# Save to Feature Store with a SageMaker Processing Job

<div class="alert alert-info"> 💡 <strong> Quick Start </strong>
To save your processed data to feature store, <strong><a style="color: #0397a7 " href="#Create-Feature-Group">
    <u>Click here to create a feature group</u></a> and follow the instruction to run a SageMaker processing job.
</strong>
</div>

This notebook uses Amazon SageMaker Feature Store (Feature Store) to create a feature group, 
executes your Data Wrangler Flow `bankdm.flow` on the entire dataset using a SageMaker 
Processing Job and ingest processed data to Feature Store. 

---

## Contents

1. [Create Feature Group](#Create-Feature-Group)
   1. [Define Feature Group](#Define-Feature-Group)
   1. [Configure Feature Group](#Configure-Feature-Group)
   1. [Initialize & Create Feature Group](#Initialize-&-Create-Feature-Group)
1. [Processing Job: Inputs and Outputs](#Inputs-and-Outputs)
1. [Run Processing Job](#Run-Processing-Job)
   1. [Job Configurations](#Job-Configurations)
   1. [Create Processing Job](#Create-Processing-Job)
   1. [Job Status & Output Location](#Job-Status-&-Output-Location)

## Create Feature Group

_What is a feature group_

A single feature corresponds to a column in your dataset. A feature group is a predefined schema for a 
collection of features - each feature in the feature group has a specified data type and name. 
A single record in a feature group corresponds to a row in your dataframe. A feature store is a 
collection of feature groups. To learn more about SageMaker Feature Store, see 
[Amazon Feature Store Documentation](http://docs.aws.amazon.com/sagemaker/latest/dg/feature-store.html).

### Define Feature Group
Select Record identifier and Event time feature name. These are required parameters for feature group
creation.
* **Record identifier name** is the name of the feature defined in the feature group's feature definitions 
whose value uniquely identifies a Record defined in the feature group's feature definitions.
* **Event time feature name** is the name of the EventTime feature of a Record in FeatureGroup. An EventTime 
is a timestamp that represents the point in time when a new event occurs that corresponds to the creation or 
update of a Record in the FeatureGroup. All Records in the FeatureGroup must have a corresponding EventTime.

<div class="alert alert-info"> 💡Record identifier and Event time feature name are required 
for feature group. After filling in the values, you can choose <b>Run Selected Cell and All Below</b> 
from the Run Menu from the menu bar. 
</div>

In [1]:
record_identifier_feature_name = "FS_ID"
if record_identifier_feature_name is None:
   raise SystemExit("Select a column name as the feature group record identifier.")

event_time_feature_name = "FS_time"
if event_time_feature_name is None:
   raise SystemExit("Select a column name as the event time feature name.")


### Feature Definitions
The following is a list of the feature names and feature types of the final dataset that will be produced 
when your data flow is used to process your input dataset. These are automatically generated from the 
step `Custom Pyspark` from `Source: Answers.Csv`. To save from a different step, go to Data Wrangler to 
select a new step to export.

<div class="alert alert-info"> 💡 <strong> Configurable Settings </strong>

1. You can select a subset of the features. By default all columns of the result dataframe will be used as 
features.
2. You can change the Data Wrangler data type to one of the Feature Store supported types 
(<b>Integral</b>, <b>Fractional</b>, or <b>String</b>). The default type is set to <b>String</b>. 
This means that, if a column in your dataset is not a <b>float</b> or <b>long</b> type, it will default 
to <b>String</b> in your Feature Store.

For <b>Event Time</b> features, make sure the format follows the feature store
<strong>
    <a style="color: #0397a7 " href="https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-quotas.html#feature-store-data-types">
    <u>Event Time feature format</u>
    </a>
</strong>
</div>

The following is a list of the feature names and data types of the final dataset that will be produced when your data flow is used to process your input dataset.

In [2]:
column_schemas = [
    {
        "name": "age",
        "type": "long"
    },
    {
        "name": "campaign",
        "type": "long"
    },
    {
        "name": "pdays",
        "type": "long"
    },
    {
        "name": "previous",
        "type": "long"
    },
    {
        "name": "no_previous_contact",
        "type": "long"
    },
    {
        "name": "not_working",
        "type": "long"
    },
    {
        "name": "FS_ID",
        "type": "long"
    },
    {
        "name": "FS_time",
        "type": "float"
    },
    {
        "name": "job_admin",
        "type": "float"
    },
    {
        "name": "job_blue-collar",
        "type": "float"
    },
    {
        "name": "job_technician",
        "type": "float"
    },
    {
        "name": "job_services",
        "type": "float"
    },
    {
        "name": "job_management",
        "type": "float"
    },
    {
        "name": "job_retired",
        "type": "float"
    },
    {
        "name": "job_entrepreneur",
        "type": "float"
    },
    {
        "name": "job_self-employed",
        "type": "float"
    },
    {
        "name": "job_housemaid",
        "type": "float"
    },
    {
        "name": "job_unemployed",
        "type": "float"
    },
    {
        "name": "job_student",
        "type": "float"
    },
    {
        "name": "job_unknown",
        "type": "float"
    },
    {
        "name": "marital_divorced",
        "type": "long"
    },
    {
        "name": "marital_married",
        "type": "long"
    },
    {
        "name": "marital_single",
        "type": "long"
    },
    {
        "name": "marital_unknown",
        "type": "long"
    },
    {
        "name": "education_basic_4y",
        "type": "long"
    },
    {
        "name": "education_basic_6y",
        "type": "long"
    },
    {
        "name": "education_basic_9y",
        "type": "long"
    },
    {
        "name": "education_high_school",
        "type": "long"
    },
    {
        "name": "education_illiterate",
        "type": "long"
    },
    {
        "name": "education_professional_course",
        "type": "long"
    },
    {
        "name": "education_university_degree",
        "type": "long"
    },
    {
        "name": "education_unknown",
        "type": "long"
    },
    {
        "name": "defaulted_no",
        "type": "long"
    },
    {
        "name": "defaulted_unknown",
        "type": "long"
    },
    {
        "name": "defaulted_yes",
        "type": "long"
    },
    {
        "name": "housing_no",
        "type": "long"
    },
    {
        "name": "housing_unknown",
        "type": "long"
    },
    {
        "name": "housing_yes",
        "type": "long"
    },
    {
        "name": "loan_no",
        "type": "long"
    },
    {
        "name": "loan_unknown",
        "type": "long"
    },
    {
        "name": "loan_yes",
        "type": "long"
    },
    {
        "name": "contact_cellular",
        "type": "long"
    },
    {
        "name": "contact_telephone",
        "type": "long"
    },
    {
        "name": "month_apr",
        "type": "long"
    },
    {
        "name": "month_aug",
        "type": "long"
    },
    {
        "name": "month_dec",
        "type": "long"
    },
    {
        "name": "month_jul",
        "type": "long"
    },
    {
        "name": "month_jun",
        "type": "long"
    },
    {
        "name": "month_mar",
        "type": "long"
    },
    {
        "name": "month_may",
        "type": "long"
    },
    {
        "name": "month_nov",
        "type": "long"
    },
    {
        "name": "month_oct",
        "type": "long"
    },
    {
        "name": "month_sep",
        "type": "long"
    },
    {
        "name": "day_of_week_fri",
        "type": "long"
    },
    {
        "name": "day_of_week_mon",
        "type": "long"
    },
    {
        "name": "day_of_week_thu",
        "type": "long"
    },
    {
        "name": "day_of_week_tue",
        "type": "long"
    },
    {
        "name": "day_of_week_wed",
        "type": "long"
    },
    {
        "name": "poutcome_failure",
        "type": "long"
    },
    {
        "name": "poutcome_nonexistent",
        "type": "long"
    },
    {
        "name": "poutcome_success",
        "type": "long"
    },
    {
        "name": "y_no",
        "type": "long"
    },
    {
        "name": "y_yes",
        "type": "long"
    }
]

Below we create the SDK input for those feature definitions. Some schema types in Data Wrangler are not 
supported by Feature Store. The following will create a default_FG_type set to String for these types.

In [3]:
from sagemaker.feature_store.feature_definition import FeatureDefinition
from sagemaker.feature_store.feature_definition import FeatureTypeEnum

default_feature_type = FeatureTypeEnum.STRING
column_to_feature_type_mapping = {
    "float": FeatureTypeEnum.FRACTIONAL,
    "long": FeatureTypeEnum.INTEGRAL
}

feature_definitions = [
    FeatureDefinition(
        feature_name=column_schema['name'], 
        feature_type=column_to_feature_type_mapping.get(column_schema['type'], default_feature_type)
    ) for column_schema in column_schemas
]

## Configure Feature Group

<div class="alert alert-info"> 💡 <strong> Configurable Settings </strong>

1. <b>feature_group_name</b>: name of the feature group.
1. <b>feature_store_offline_s3_uri</b>: SageMaker FeatureStore writes the data in the OfflineStore of a FeatureGroup to a S3 location owned by you.
1. <b>enable_online_store</b>: controls if online store is enabled. Enabling the online store allows quick access to the latest value for a Record via the GetRecord API.
1. <b>iam_role</b>: IAM role for executing the processing job.
</div>

In [4]:
from time import gmtime, strftime
import uuid
import sagemaker 

# Sagemaker session
sess = sagemaker.Session()

# You can configure this with your own bucket name, e.g.
# bucket = <my-own-storage-bucket>
bucket = sess.default_bucket()

# IAM role for executing the processing job.
iam_role = sagemaker.get_execution_role()

# flow name and an unique ID for this export (used later as the processing job name for the export)
flow_name = "bankdm"
flow_export_id = f"{strftime('%d-%H-%M-%S', gmtime())}-{str(uuid.uuid4())[:8]}"
flow_export_name = f"flow-{flow_export_id}"

# feature group name, with flow_name and an unique id. You can give it a customized name
feature_group_name = f"FG-{flow_name}-{str(uuid.uuid4())[:8]}"
print(f"Feature Group Name: {feature_group_name}")

# SageMaker FeatureStore writes the data in the OfflineStore of a FeatureGroup to a 
# S3 location owned by you.
feature_store_offline_s3_uri = 's3://' + bucket

# controls if online store is enabled. Enabling the online store allows quick access to 
# the latest value for a Record via the GetRecord API.
enable_online_store = False

Feature Group Name: FG-bankdm-bca79e26


### Initialize & Create Feature Group

In [5]:
# Initialize Boto3 session that is required to create feature group
import boto3
from sagemaker.session import Session

region = boto3.Session().region_name
boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

Feature group is initialized and created below

In [6]:
from sagemaker.feature_store.feature_group import FeatureGroup

feature_group = FeatureGroup(
    name=feature_group_name, sagemaker_session=feature_store_session, feature_definitions=feature_definitions)

feature_group.create(
    s3_uri=feature_store_offline_s3_uri,
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=iam_role,
    enable_online_store=enable_online_store
)

{'FeatureGroupArn': 'arn:aws:sagemaker:ap-southeast-1:138604873012:feature-group/fg-bankdm-bca79e26',
 'ResponseMetadata': {'RequestId': 'fd367f8e-4c65-44ab-90fc-dd095b3b99cb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fd367f8e-4c65-44ab-90fc-dd095b3b99cb',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '100',
   'date': 'Sun, 10 Oct 2021 01:53:49 GMT'},
  'RetryAttempts': 0}}

Invoke the Feature Store API to create the feature group and wait until it is ready

In [7]:
import time
def wait_for_feature_group_creation_complete(feature_group):
    """Helper function to wait for the completions of creating a feature group"""
    response = feature_group.describe()
    status = response.get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        response = feature_group.describe()
        status = response.get("FeatureGroupStatus")

    if status != "Created":
        print(f"Failed to create feature group, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create feature group {feature_group.name}, status: {status}, reason: {failureReason}"
        )
    print(f"FeatureGroup {feature_group.name} successfully created.")

wait_for_feature_group_creation_complete(feature_group=feature_group)

Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
FeatureGroup FG-bankdm-bca79e26 successfully created.


Now that the feature group is created, You will use a processing job to process your 
        data at scale and ingest the transformed data into this feature group.

# Inputs and Outputs

The below settings configure the inputs and outputs for the flow export.

<div class="alert alert-info"> 💡 <strong> Configurable Settings </strong>

In <b>Input - Source</b> you can configure the data sources that will be used as input by Data Wrangler

1. For S3 sources, configure the source attribute that points to the input S3 prefixes
2. For all other sources, configure attributes like query_string, database in the source's 
<b>DatasetDefinition</b> object.

If you modify the inputs the provided data must have the same schema and format as the data used in the Flow. 
You should also re-execute the cells in this section if you have modified the settings in any data sources.
</div>

In [8]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.dataset_definition.inputs import AthenaDatasetDefinition, DatasetDefinition, RedshiftDatasetDefinition

data_sources = []

## Input - Redshift Source: dm-data

In [9]:
data_sources.append(ProcessingInput(
    input_name="dm-data",
    # You can override below to point to other database or use different queries
    dataset_definition=DatasetDefinition(
        local_path="/opt/ml/processing/dm-data",
        data_distribution_type="FullyReplicated",
        redshift_dataset_definition=RedshiftDatasetDefinition(
            cluster_id="bankdm",
            database="bankdm",
            db_user="bankdm",
            query_string="SELECT * FROM( select * from dm.data )",
            cluster_role_arn="arn:aws:iam::138604873012:role/BankDM-RedShift",
            output_s3_uri="s3://sagemaker-ap-southeast-1-138604873012/redshift/dm-data/",
            output_format="PARQUET"
        )
    )
))

### Output: Feature Store 

Below are the inputs required by the SageMaker Python SDK to launch a processing job with feature store as an output.

In [10]:
from sagemaker.processing import FeatureStoreOutput

# Output name is auto-generated from the select node's ID + output name from the flow file.
output_name = "9331c414-3065-4470-8184-0bb846f77ecb.default"

processing_job_output = ProcessingOutput(
    output_name=output_name,
    app_managed=True,
    feature_store_output=FeatureStoreOutput(feature_group_name=feature_group_name),
)

## Upload Flow to S3

To use the Data Wrangler as an input to the processing job,  first upload your flow file to Amazon S3.

In [11]:
import os
import json
import boto3

# name of the flow file which should exist in the current notebook working directory
flow_file_name = "bankdm.flow"

# Load .flow file from current notebook working directory 
!echo "Loading flow file from current notebook working directory: $PWD"

with open(flow_file_name) as f:
    flow = json.load(f)

# Upload flow to S3
s3_client = boto3.client("s3")
s3_client.upload_file(flow_file_name, bucket, f"data_wrangler_flows/{flow_export_name}.flow", ExtraArgs={"ServerSideEncryption": "aws:kms"})

flow_s3_uri = f"s3://{bucket}/data_wrangler_flows/{flow_export_name}.flow"

print(f"Data Wrangler flow {flow_file_name} uploaded to {flow_s3_uri}")

Loading flow file from current notebook working directory: /root/bankdm-p-2xq3nytjxg4p/sagemaker-bankdm-p-2xq3nytjxg4p-modelbuild
Data Wrangler flow bankdm.flow uploaded to s3://sagemaker-ap-southeast-1-138604873012/data_wrangler_flows/flow-10-01-53-49-5bbf083e.flow


The Data Wrangler Flow is also provided to the Processing Job as an input source which we configure below.

In [12]:
## Input - Flow: bankdm.flow
flow_input = ProcessingInput(
    source=flow_s3_uri,
    destination="/opt/ml/processing/flow",
    input_name="flow",
    s3_data_type="S3Prefix",
    s3_input_mode="File",
    s3_data_distribution_type="FullyReplicated"
)

# Run Processing Job 
## Job Configurations

<div class="alert alert-info"> 💡 <strong> Configurable Settings </strong>

You can configure the following settings for Processing Jobs. If you change any configurations you will 
need to re-execute this and all cells below it by selecting the Run menu above and click 
<b>Run Selected Cells and All Below</b>

1. IAM role for executing the processing job. 
2. A unique name of the processing job. Give a unique name every time you re-execute processing jobs
3. Data Wrangler Container URL.
4. Instance count, instance type and storage volume size in GB.
5. Content type for each output. Data Wrangler supports CSV as default and Parquet.
6. Network Isolation settings
7. KMS key to encrypt output data
</div>

In [13]:
# IAM role for executing the processing job.
iam_role = sagemaker.get_execution_role()

# Unique processing job name. Give a unique name every time you re-execute processing jobs
processing_job_name = f"data-wrangler-flow-processing-{flow_export_id}"

# Data Wrangler Container URL.
container_uri = "119527597002.dkr.ecr.ap-southeast-1.amazonaws.com/sagemaker-data-wrangler-container:1.x"
# Pinned Data Wrangler Container URL. 
container_uri_pinned = "119527597002.dkr.ecr.ap-southeast-1.amazonaws.com/sagemaker-data-wrangler-container:1.11.0"

# Processing Job Instance count and instance type.
instance_count = 2
instance_type = "ml.m5.4xlarge"

# Size in GB of the EBS volume to use for storing data during processing
volume_size_in_gb = 30

# Content type for each output. Data Wrangler supports CSV as default and Parquet.
output_content_type = "CSV"

# Network Isolation mode; default is off
enable_network_isolation = False

# Output configuration used as processing job container arguments 
output_config = {
    output_name: {
        "content_type": output_content_type
    }
}

# KMS key for per object encryption; default is None
kms_key = None

## Create Processing Job

To launch a Processing Job, you will use the SageMaker Python SDK to create a Processor function.

In [14]:
from sagemaker.processing import Processor
from sagemaker.network import NetworkConfig

processor = Processor(
    role=iam_role,
    image_uri=container_uri,
    instance_count=instance_count,
    instance_type=instance_type,
    volume_size_in_gb=volume_size_in_gb,
    network_config=NetworkConfig(enable_network_isolation=enable_network_isolation),
    sagemaker_session=sess,
    output_kms_key=kms_key
)

# Start Job
processor.run(
    inputs=[flow_input] + data_sources, 
    outputs=[processing_job_output],
    arguments=[f"--output-config '{json.dumps(output_config)}'"],
    wait=False,
    logs=False,
    job_name=processing_job_name
)


Job Name:  data-wrangler-flow-processing-10-01-53-49-5bbf083e
Inputs:  [{'InputName': 'flow', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-southeast-1-138604873012/data_wrangler_flows/flow-10-01-53-49-5bbf083e.flow', 'LocalPath': '/opt/ml/processing/flow', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'dm-data', 'AppManaged': False, 'DatasetDefinition': {'LocalPath': '/opt/ml/processing/dm-data', 'DataDistributionType': 'FullyReplicated', 'RedshiftDatasetDefinition': {'ClusterId': 'bankdm', 'Database': 'bankdm', 'DbUser': 'bankdm', 'QueryString': 'SELECT * FROM( select * from dm.data )', 'ClusterRoleArn': 'arn:aws:iam::138604873012:role/BankDM-RedShift', 'OutputS3Uri': 's3://sagemaker-ap-southeast-1-138604873012/redshift/dm-data/', 'OutputFormat': 'PARQUET'}}}]
Outputs:  [{'OutputName': '9331c414-3065-4470-8184-0bb846f77ecb.default', 'AppManaged': True, 'FeatureStoreOutput': {'

## Job Status & S3 Output Location

Below you wait for processing job to finish. If it finishes successfully, your feature group should be populated 
with transformed feature values. In addition the raw parameters used by the Processing Job will be printed.

In [15]:
job_result = sess.wait_for_processing_job(processing_job_name)
job_result

............................................................................!

{'ProcessingInputs': [{'InputName': 'flow',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-ap-southeast-1-138604873012/data_wrangler_flows/flow-10-01-53-49-5bbf083e.flow',
    'LocalPath': '/opt/ml/processing/flow',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}},
  {'InputName': 'dm-data',
   'AppManaged': False,
   'DatasetDefinition': {'RedshiftDatasetDefinition': {'ClusterId': 'bankdm',
     'Database': 'bankdm',
     'DbUser': 'bankdm',
     'QueryString': 'SELECT * FROM( select * from dm.data )',
     'ClusterRoleArn': 'arn:aws:iam::138604873012:role/BankDM-RedShift',
     'OutputS3Uri': 's3://sagemaker-ap-southeast-1-138604873012/redshift/dm-data/',
     'OutputFormat': 'PARQUET'},
    'LocalPath': '/opt/ml/processing/dm-data',
    'DataDistributionType': 'FullyReplicated',
    'InputMode': 'File'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': '9331c414-3065-4

You can view newly created feature group in Studio, refer to [Use Amazon SageMaker Feature Store with Amazon SageMaker Studio](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-use-with-studio.html)
for detailed guide.[Learn more about SageMaker Feature Store](https://github.com/aws/amazon-sagemaker-examples/tree/master/sagemaker-featurestore)